In [16]:
import pandas as pd
import string 
df_reviews = pd.read_csv('IMDB_Dataset.csv')

df_reviews_teste = df_reviews.sample(97)

df_filtrado_positivo = df_reviews_teste[df_reviews_teste['sentiment']=='positive']
df_filtrado_negativo = df_reviews_teste[df_reviews_teste['sentiment']=='negative']


dict_palavras_pos={}
for review in df_filtrado_positivo['review']:
    review=review.replace('<br />', " ")
    translator= str.maketrans(' ',' ', string.punctuation)
    review = review.translate(translator)
    review=set(review.lower().split())
    for palavra in review:
        if palavra in dict_palavras_pos:
            dict_palavras_pos[palavra]+=1
        else:
            dict_palavras_pos[palavra]=1


dict_palavras_neg={}
for review in df_filtrado_negativo['review']:
    review=review.replace('<br />', " ")
    translator= str.maketrans(' ',' ', string.punctuation)
    review = review.translate(translator)
    review=set(review.lower().split())
    for palavra in review:
        palavra=palavra.lower()
        if palavra in dict_palavras_neg:
            dict_palavras_neg[palavra]+=1
        else:
            dict_palavras_neg[palavra]=1



In [22]:
#calcular P(palavra|classe)

#positivo
quant_review_pos = len(df_filtrado_positivo)
dict_prob_dado_pos = {}

for palavra, quant in dict_palavras_pos.items():
    prob = quant/quant_review_pos
    dict_prob_dado_pos[palavra] = prob



#negativo
quant_review_neg = len(df_filtrado_negativo)
dict_prob_dado_neg = {}

for palavra, quant in dict_palavras_neg.items():
    prob = quant/quant_review_neg
    dict_prob_dado_neg[palavra] = prob



#calcular P(nao palavra|classe)

#positvo
dict_prob_n_palavra_dado_pos = {}
for palavra, prob in dict_prob_dado_pos.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_pos[palavra] = prob_n_palavra

#negativo
dict_prob_n_palavra_dado_neg = {}
for palavra, prob in dict_prob_dado_neg.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_neg[palavra] = prob_n_palavra

2929 2929


In [25]:
#Calcular P(palavra)
palavras_totais ={}

for palavra, quant in dict_palavras_pos.items():
    palavras_totais[palavra] = quant
for palavra, quant in dict_palavras_neg.items():
    if palavra in palavras_totais:
        palavras_totais[palavra]+=quant
    else:
        palavras_totais[palavra]=quant

prob_palavras_dict={}
for palavra in palavras_totais:
    p_palavra=palavras_totais[palavra]/len(df_reviews_teste)
    prob_palavras_dict[palavra]=p_palavra

4677 2929 2889


In [39]:
#Calcular P(classe|texto)

p_negativo = 0.5
p_positivo = 0.5
p_total = 1

# for texto in df_reviews_teste['review']:
texto=df_reviews_teste['review']
texto=texto.iloc[1]


#positivo
for palavra in palavras_totais:
    if palavra in texto:
        if palavra in dict_prob_dado_pos:
            p_total *= dict_prob_dado_pos[palavra]/prob_palavras_dict[palavra]
    else:
        if palavra in dict_prob_n_palavra_dado_pos:
            p_total *= dict_prob_n_palavra_dado_pos[palavra]/(1-prob_palavras_dict[palavra])
p_total*=p_positivo

print(p_total,df_reviews_teste['sentiment'].iloc[1])

0.5128122715351672 positive
